# CS246 - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4 MB 71 kB/s 
     |████████████████████████████████| 198 kB 44.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7a09df8b70796788e31793da16726d13954711cfc2a71681c50b63309a124239
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
# Import pydrive for login Google Cloud SDK
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# Download file products.csv
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

# Download file order_products__train.csv
id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [4]:
# Import lib for table data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import PySpark
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-09-27 05:29:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  6.07MB/s    in 2.2s    

2021-09-27 05:29:35 (6.07 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [8]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [9]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [10]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [14]:
# YOUR CODE HERE
transactions = orders.join(products, on='product_id').groupby('order_id').agg(collect_list('product_name').alias('product_names'))
transactions.take(10)

[Row(order_id=1342, product_names=['Bag of Organic Bananas', 'Seedless Cucumbers', 'Organic Mandarins', 'Organic Strawberries', 'Versatile Stain Remover', 'Pink Lady Apples', 'Chicken Apple Sausage', 'Raw Shrimp']),
 Row(order_id=1591, product_names=['Cracked Wheat', 'Organic Greek Whole Milk Blended Vanilla Bean Yogurt', 'Navel Oranges', 'Spinach', 'Original Patties (100965) 12 Oz Breakfast', 'Cinnamon Multigrain Cereal', 'Chewy 25% Low Sugar Chocolate Chip Granola', 'Uncured Genoa Salami', 'Natural Vanilla Ice Cream', 'Lemon Yogurt', 'Strawberry Rhubarb Yoghurt', 'Garlic', 'Pure Vanilla Extract', 'Lower Sugar Instant Oatmeal  Variety', 'Organic Bunny Fruit Snacks Berry Patch', 'Buttermilk Waffles', 'Granny Smith Apples', 'Medium Scarlet Raspberries', 'Banana', 'Strawberry Banana Smoothie', 'Green Machine Juice Smoothie', 'Ultra Thin Sliced Provolone Cheese', 'Oven Roasted Turkey Breast', 'Original Turkey Burgers Smoke Flavor Added', 'Original Whole Grain Chips', 'Goldfish Pretzel Bak

In [15]:
transactions.show()

+--------+--------------------+
|order_id|       product_names|
+--------+--------------------+
|    1342|[Bag of Organic B...|
|    1591|[Cracked Wheat, O...|
|    4519|[Beet Apple Carro...|
|    4935|             [Vodka]|
|    6357|[Fresh Mozzarella...|
|   10362|[Crinkle Cut Fren...|
|   19204|[Lemon Lime Thirs...|
|   29601|[Toasted Coconut ...|
|   31035|[Bag of Organic B...|
|   40011|[Sea Salt Macadam...|
|   46266|[Organic Uncured ...|
|   51607|[Major Dickason's...|
|   58797|[Bag of Organic B...|
|   61793|[Raspberries, Nat...|
|   67089|[Banana, Organic ...|
|   70863|[Bathroom Tissue,...|
|   88674|[Unsweetened Almo...|
|   91937|[No. 485 Gin, Tra...|
|   92317|[Banana, Indian M...|
|   99621|[Organic Basil, F...|
+--------+--------------------+
only showing top 20 rows



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [16]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth

# Hyperparameter
minSupport=0.01
minConfidence=0.5

# Initthe FP-Growth algorithm 
fpGrowth = FPGrowth(itemsCol="product_names", minSupport=minSupport, minConfidence=minConfidence)

# Fitting model
model = fpGrowth.fit(transactions)

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [17]:
# YOUR CODE HERE
model.freqItemsets.show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Strawber...| 3074|
|[Organic Strawber...| 2174|
|[Organic Baby Spi...| 9784|
|[Organic Baby Spi...| 1639|
|[Organic Baby Spi...| 2236|
|[Organic Baby Spi...| 2000|
|       [Large Lemon]| 8135|
|[Large Lemon, Ban...| 2158|
|   [Organic Avocado]| 7409|
|[Organic Avocado,...| 1349|
|[Organic Avocado,...| 1402|
|[Organic Avocado,...| 2216|
|[Organic Hass Avo...| 7293|
|[Organic Hass Avo...| 1539|
|[Organic Hass Avo...| 2420|
|      [Strawberries]| 6494|
|[Strawberries, Ba...| 1948|
+--------------------+-----+
only showing top 20 rows



In [18]:
model.associationRules.show()

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



In [19]:
print("Couting Frequent Itemsets: {}".format(model.freqItemsets.count()))

Couting Frequent Itemsets: 120


In [20]:
print("Couting Association Rules: {}".format(model.associationRules.count()))

Couting Association Rules: 0


Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [21]:
# YOUR CODE HERE

# Hyperparameter
newMinSupport=0.001

# Initthe FP-Growth algorithm 
fpGrowth = FPGrowth(itemsCol="product_names", minSupport=newMinSupport, minConfidence=minConfidence)

# Fitting model
model = fpGrowth.fit(transactions)

In [22]:
print("Couting Frequent Itemsets: {}".format(model.freqItemsets.count()))
print("Couting Association Rules: {}".format(model.associationRules.count()))

Couting Frequent Itemsets: 4444
Couting Association Rules: 11


To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [ ]:
# YOUR CODE HERE

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!